In [1]:
#загружаем необходимые библиотеки
import pandas as pd
import json
import requests
import time
import traceback
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory

In [2]:
!python --version

Python 3.8.8


In [3]:
print('pandas ', pd.__version__)
print('json ', json.__version__)
print('requests ', requests.__version__)
print('time ', time.__name__)
print('traceback ', traceback.__name__)
print('os ', os.__name__)
print('tkinter ', Tk.__module__)

pandas  1.2.4
json  2.0.9
requests  2.25.1
time  time
traceback  traceback
os  os
tkinter  tkinter


In [4]:
#функция для сохранения результата запросов IP NUMBER
def results_save(js_res):
    number = []
    name = []
    exe_prod = []
    details = []
    subj = []
    depart = []
    bailiff = []
    ip_end =[]
    
    for i in js_res['response']['result']:
# обработка не найденных в банке данных ИП  (вероятно фиктивные номера или введенные с ошибкой) надо объединить эти условия
        if i['result'] == [] or i['result'] == None:
                number.append((i['query']['params'])['number'])
                name.append('empty')
                exe_prod.append('empty')
                details.append('empty')
                subj.append('empty')
                depart.append('empty')
                bailiff.append('empty')
                ip_end.append('empty')
        
#обработка номеров ИП имеющих заведомо некорректный формат
        elif i['result'] == {'code': 0, 'message': 'The given data was invalid.', 'errors': {'number': ['Поле number имеет ошибочный формат.']}}:
            number.append((i['query']['params'])['number'])
            name.append('Поле number имеет ошибочный формат.')
            exe_prod.append('Поле number имеет ошибочный формат.')
            details.append('Поле number имеет ошибочный формат.')
            subj.append('Поле number имеет ошибочный формат.')
            depart.append('Поле number имеет ошибочный формат.')
            bailiff.append('Поле number имеет ошибочный формат.')
            ip_end.append('Поле number имеет ошибочный формат.')      
        else:
# обработка найденных ИП
            if len(i['result']) == 1:
                for j in (i['result']):
                    number.append((i['query']['params'])['number'])
                    name.append(str((j)['name']))
                    exe_prod.append((j)['exe_production'])
                    details.append((j)['details'])
                    subj.append((j)['subject'])
                    depart.append((j)['department'])
                    bailiff.append((j)['bailiff'])
                    ip_end.append((j)['ip_end'])
            else:
                #корректировка списка number для компенсации дублей
                qnt = 0
                while qnt < (len(i['result'])):
                    number.append((i['query']['params'])['number'])
                    qnt += 1
                
                for dbl in (i['result']):
                    name.append((dbl)['name'])
                    exe_prod.append((dbl)['exe_production'])
                    details.append((dbl)['details'])
                    subj.append((dbl)['subject'])
                    depart.append((dbl)['department'])
                    bailiff.append((dbl)['bailiff'])
                    ip_end.append((dbl)['ip_end'])
                                
#    сборка результатов                 
    query_results = pd.DataFrame({'number' : number, 'name' : name,'exe_prod' : exe_prod,'details' : details,'subj' : subj,'depart' : bailiff,'ip_end' : ip_end})
    return query_results

In [5]:
#выбор файла с входными данными
root = Tk()
filename = askopenfilename()
print(filename)

#выбор целевого каталога (туда мы сохраним результат)
target_path = askdirectory(title = 'Выбор каталога для выгрузки результата',initialdir = os.path.expanduser(u'C://'))
target_path = target_path.replace(r"/","\\")
root.destroy()

D:/work/FSSP/ip.xlsx


In [6]:
#загружаем входные данные
ip = pd.read_excel(filename)
# total_list = ip['ip'].to_list()
print('загружено ',len(ip),' строк')

загружено  309  строк


In [7]:
#разметка входных данных по "батчам" - по 50 клиентов (ограничение API)
ip['batch'] = ip.index//50 + 1

In [9]:
#основной модуль

#инициализация результирующего датафрейма, инициализвция переменной, в которой хранится ключ доступа
summary = pd.DataFrame()
token = '***********'

#--------------------------------------------------------> эта часть должна быть вынесена в отдельную функцию, их будет три вида в зависимости от типа запроса, подгружаться должна будет выбранная опцию
#формирование входного JSON объекта
for i in list(ip['batch'].unique()):
    dic = {}
    rq = []
    js = ''
    print('Сессия - ',i,'\n')
    for j in ip[ip['batch'] == i].index:
        rq.append({'type':3,'params':{'number':ip['ip'][j]}})
    dic = {'token':token,'request':rq}
    js = json.dumps(dic)
#     print(rq)
#-------------------------------------------------------->    

#отправка группового запроса
##перед отправкой отчета неплохо бы проверить статус предыдущей задачи
    headers = {'accept': 'application/json','Content-Type': 'application/json'}

    try:
        response = requests.post('https://api-ip.fssp.gov.ru/api/v1.0/search/group', headers=headers, data=js)
        js_task = json.loads(response.content)
        print(js_task)
    except:
        print(sys.exc_info())

    
    while js_task['exception'] == 'Дождитесь результата предыдущего группового запроса':
        time.sleep(20)
        response = requests.post('https://api-ip.fssp.gov.ru/api/v1.0/search/group', headers=headers, data=js)
        js_task = json.loads(response.content)
    
#отправка запросов о статусе задачи и о выгрузке результата
    headers = {'accept': 'application/json'}
    params = (('token', token),('task', (js_task['response'])['task']))
    
##запрос статуса задачи - надо сделать отдельной функцией
    status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
    js_status = json.loads(status.content)
    print(js_status)
    time.sleep(3)
    
##надо переделать при наступлении третьего статуса все вывалится в ошибку / возможно эта проверка вообще избыточна
    while (js_status['code'] != 0) and (js_status['response']['progress']) != '50 of 50':
        time.sleep(20)
        status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
        js_status = json.loads(status.content)
        print(js_status['code'])
        print(js_status['response']['progress'])

##получение результатов запроса
    try:
        response1 = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/result', headers=headers, params=params)
        js_res = json.loads(response1.content)
    except:
        print(sys.exc_info())
    time.sleep(3)
    
    while (js_res['code'] != 2) and (js_res['response']['task_end'] is None):
        time.sleep(20)
        response1 = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/result', headers=headers, params=params)
        js_res = json.loads(response1.content)

#этот блок обработки ответов на запрос (надо доработать)
#     if js_res['code'] == 0:
#         print('keep go on')
#         #put save function
#     if js_res['code'] == 3:
#         print('error')
#         #print errors reason
#     print('next','\n')
    
#проверка полноты отработки группового запроса
    status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
    js_status = json.loads(status.content)
    print(js_status['response']['progress'],'\n')  

#сохранение ответа на запрос в результирующий датафрейм
    summary = summary.append(results_save(js_res))

#выгрузка результатов в файл в ранее выбранный каталог
summary.to_excel(os.path.join(target_path,'summary_result_test1.xlsx'))
print('Выгружено ',len(summary),' строк')

Сессия -  1 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '77cd00b1-39bb-4681-bb6b-3361afa18944'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  2 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '8ed66957-f7e5-44fd-852d-65095ec233c2'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  3 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '31b22592-d4bc-47cc-bfc5-7f9c3532bb32'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  4 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': 'd58d494d-9f87-4abf-b9b0-a0ab337087e1'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 2, 'progress': '0 of 50'}}
50 of 50 

Сессия -  5 

{'status': 'success', 'code': 0, 'exce